In [0]:
import numpy as np
import pandas as pd

import tensorflow as tf

from skimage.io import imread, imshow
from pathlib import Path

In [0]:
print(tf.VERSION)
print(tf.test.is_gpu_available())

1.12.0
True


In [0]:
np.random.seed(42)
tf.random.set_random_seed(42)

## Download and extract Cifar-10 dataset

In [0]:
#!wget 'https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz'

In [0]:
#!tar -xvzf cifar-10-binary.tar.gz

## Define constants

In [0]:
_HEIGHT = 32
_WIDTH = 32
_NUM_CHANNELS = 3
_DEFAULT_IMAGE_BYTES = _HEIGHT * _WIDTH * _NUM_CHANNELS
# The record is the image plus a one-byte label
_RECORD_BYTES = _DEFAULT_IMAGE_BYTES + 1
_NUM_CLASSES = 10
_NUM_DATA_FILES = 5

_NUM_IMAGES = {
    'train': 50000,
    'validation': 10000,
}
DATASET_NAME = 'CIFAR-10'

## Data Preprocessing

In [0]:
def get_fnames(train=True):
  """Returns a list of file paths for either train or test"""
  data_path = Path('.') / 'cifar-10-batches-bin'
  if train:
    return [str(f) for f in sorted(list(data_path.glob('data*.bin')))]
  else:
    return [str(data_path / 'test_batch.bin')]

In [0]:
def parse_record(raw_record, is_training, dtype):
  """Parse CIFAR-10 image and label from a raw record."""
  # Convert bytes to a vector of uint8 that is record_bytes long.
  record_vector = tf.decode_raw(raw_record, tf.uint8)

  # The first byte represents the label, which we convert from uint8 to int32 and then to one-hot.
  label = tf.cast(record_vector[0], tf.int32)

  # The remaining bytes after the label represent the image, which we reshape from [depth * height * width] to [depth, height, width].
  depth_major = tf.reshape(record_vector[1:_RECORD_BYTES], [_NUM_CHANNELS, _HEIGHT, _WIDTH])

  # Convert from [depth, height, width] to [height, width, depth], and cast as float32.
  image = tf.cast(tf.transpose(depth_major, [1, 2, 0]), tf.float32)

  image = preprocess_image(image, is_training)
  image = tf.cast(image, dtype)
  return image, label

In [0]:
def preprocess_image(image, is_training):
  """Preprocess a single image of layout [height, width, depth]."""
  if is_training:
    # Resize the image to add four extra pixels on each side.
    image = tf.image.resize_image_with_crop_or_pad(image, _HEIGHT + 8, _WIDTH + 8)

    # Randomly crop a [_HEIGHT, _WIDTH] section of the image.
    image = tf.random_crop(image, [_HEIGHT, _WIDTH, _NUM_CHANNELS])

    # Randomly flip the image horizontally.
    image = tf.image.random_flip_left_right(image)

  # Subtract off the mean and divide by the variance of the pixels.
  image = tf.image.per_image_standardization(image)
  return image

In [0]:
def process_record_dataset(dataset, is_train, batch_size, shuffle_buffer, num_epochs=1, dtype=tf.float32,
                           datasets_num_private_threads=None, num_parallel_batches=1):
  dataset = dataset.prefetch(buffer_size=batch_size)
  if is_train:
    dataset= dataset.shuffle(buffer_size=_NUM_IMAGES['train'])
  dataset = dataset.repeat(num_epochs)
  dataset = dataset.apply(
      tf.contrib.data.map_and_batch(
        lambda value: parse_record(value, is_train, dtype),
        batch_size=batch_size,
        num_parallel_batches=num_parallel_batches,
        drop_remainder=False))
  dataset = dataset.prefetch(buffer_size=tf.contrib.data.AUTOTUNE)
  
  if datasets_num_private_threads:
    tf.logging.info('datasets_num_private_threads: %s',
                    datasets_num_private_threads)
    dataset = threadpool.override_threadpool(
        dataset,
        threadpool.PrivateThreadPool(
            datasets_num_private_threads,
            display_name='input_pipeline_thread_pool'))

  return dataset

In [0]:
fnames_train = get_fnames(train=True)
dataset_train = tf.data.FixedLengthRecordDataset(fnames_train, _RECORD_BYTES)

fnames_test = get_fnames(train=False)
dataset_test = tf.data.FixedLengthRecordDataset(fnames_test, _RECORD_BYTES)

## Model